In [1]:
from functools import wraps
 
def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction
 
@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
 
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


In [2]:
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


In [3]:
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated
 
@decorator_name  # 包一层的直接函数名
def func():
    return("Function is running")
 
can_run = True
print(func())
# Output: Function is running
 
can_run = False
print(func())
# Output: Function will not run

Function is running
Function will not run


In [4]:
def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator  # 返回装饰器
 
@logit()  # 包两层的要调用下
def myfunc1():
    pass
 
myfunc1()
# Output: myfunc1 was called
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串
 
@logit(logfile='func2.log')
def myfunc2():
    pass
 
myfunc2()
# Output: myfunc2 was called
# 现在一个叫做 func2.log 的文件出现了，里面的内容就是上面的字符串

myfunc1 was called
myfunc2 was called


In [2]:
import tensorflow as tf

In [3]:
import functools

In [4]:
tf.__version__

'1.15.0'

In [5]:
tf.get_default_graph().get_name_scope()

''

In [6]:
with tf.name_scope('V1'):  
    # a1 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))  
    a2 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')  
with tf.name_scope('V2'):  
    # a3 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))  
    a4 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')  
    print(tf.get_default_graph().get_name_scope())
    
with tf.Session() as sess:  
    sess.run(tf.initialize_all_variables())  
    # print a1.name  
    print(a2.name)  
    # print a3.name  
    print(a4.name)  

V2
Instructions for updating:
Use `tf.global_variables_initializer` instead.
V1/a2:0
V2/a2:0


In [7]:
tf.get_default_graph().get_name_scope()

''

In [8]:
class Meta(type):  
   
    record_cls = []

    def __new__(cls, name, bases, attrs):
        new_cls = type.__new__(cls, name, bases, attrs)
        Meta.record_cls.append(new_cls)
        return new_cls
    
    def __init__(cls, name, bases, namespace, **kwargs):
        super().__init__(name, bases, namespace, **kwargs)
        if not hasattr(cls, 'registory'):  # build only in the first subclass
            # this is the base class
            cls.registory = {}
        else:
            # this is the subclass
            cls.registory[name.lower()] = cls
            
    def create(cls, class_name, *args, **kwargs):
        return cls.registory[class_name.lower()](*args, **kwargs)


In [248]:
class Module(object, metaclass=Meta):
    
    def __init__(self, name, **kwargs):
        self.name = name+'qq'
        
        self.parent_name = tf.get_default_graph().get_name_scope()
        if self.parent_name:
            self.total_name = self.parent_name + '/' + self.name + '/'
        else:
            self.total_name = self.name + '/'
        self._ns_call_count = 0

        self.name_scope = tf.name_scope(self.total_name)
        self.restore_variables = {}
        self.summaries = []
        self.updates = []
        print('====',self.name,name,self._ns_call_count)

    @staticmethod
    def with_name_scope(func):
        
        @functools.wraps(func)
        def inner(self, *args, **kwargs):
            parent_name = tf.get_default_graph().get_name_scope()
            name = self.name + '_-_' + str(self._ns_call_count) if self._ns_call_count else self.name
            if parent_name:
                total_name = parent_name + '/' + name + '/'
            else:
                total_name = name + '/'
            self._ns_call_count += 1
            print('inner',self._ns_call_count,total_name,parent_name,name)
            with tf.name_scope(total_name):
                return func(self, *args, **kwargs)

        return inner

    def _get_recursive_list(self, key, seen):
        print('=======',self.name)
        ret = []

        if self in seen:
            return ret
        seen.add(self)
        ret.extend(getattr(self, key))
        print('ret:',ret,'key',key)
        def _extend(val):
            if isinstance(val, Module):
                ret.extend(val._get_recursive_list(key, seen))
            if isinstance(val, tf.keras.layers.Layer):
                if not (val in seen):
                    ret.extend(getattr(val, key))
                seen.add(val)

        for attr in dir(self):
            print('attr',attr)
            val = getattr(self, attr)
            if isinstance(val, list):
                for v in val:
                    _extend(v)
            elif isinstance(val, dict):
                for k, v in val.items():
                    _extend(v)
            else:
                _extend(val)

        return ret

    def get_updates(self, seen=None):
        if not isinstance(seen, set):
            seen = set()
        print('get_updates', seen)
        return self._get_recursive_list('updates', seen)

In [249]:
class Dense(tf.Module):

    def __init__(self, name,):
        super(Dense, self).__init__(name)
        with self.name_scope:
            print('+',name,tf.get_default_graph().get_name_scope(),self.name_scope)
            self.w = tf.Variable(
                tf.random_normal(shape=[2,3], mean=0, stddev=1),
                name='w', 
                dtype=tf.float32)
            self.b = tf.Variable(
                tf.random_normal(shape=[2,3], mean=0, stddev=1),
                name='b',
                dtype=tf.float32)

    @tf.Module.with_name_scope
    def __call__(self, training=False):
        print(tf.get_default_graph().get_name_scope())
        self.de = tf.Variable(tf.random.normal([4, 3]))
        return training

In [250]:
e=Dense('ee')
e

+ ee ee_4 <tensorflow.python.framework.ops.name_scope object at 0x0000018EE8305CF8>


In [251]:
e()

ee_4


False

In [252]:
class MLP(tf.Module):

    def __init__(self, name):
        super(MLP, self).__init__(name)
        with self.name_scope:
            print('*',name,tf.get_default_graph().get_name_scope(),self.name_scope)
            self.layers = []
            for i in range(2):
                self.layers.append(
                    Dense(
                        'dense_%d' % (i,),
                        )
                    )
        print(dir(self))
    @tf.Module.with_name_scope
    def __call__(self,training=False):
        print(tf.get_default_graph().get_name_scope())
        self.ml = tf.Variable(tf.random.normal([4, 3]))
        return 1

In [253]:
a = MLP('mlp')

* mlp mlp_7 <tensorflow.python.framework.ops.name_scope object at 0x0000018EE830AF28>
+ dense_0 mlp_7/dense_0 <tensorflow.python.framework.ops.name_scope object at 0x0000018EE830AC18>
+ dense_1 mlp_7/dense_1 <tensorflow.python.framework.ops.name_scope object at 0x0000018EE831FFD0>
['_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_variable_with_custom_getter', '_checkpoint_dependencies', '_deferred_dependencies', '_flatten', '_gather_saveables_for_checkpoint', '_handle_deferred_dependencies', '_list_extra_dependencies_for_serialization', '_list_functions_for_serialization', '_lookup_dependency', '_maybe_initialize_trackable', '_name', '_name_based

In [254]:
a()
a.trainable_variables

mlp_7


(<tf.Variable 'mlp_7/Variable:0' shape=(4, 3) dtype=float32_ref>,
 <tf.Variable 'mlp_7/dense_0/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp_7/dense_0/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp_7/dense_1/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp_7/dense_1/w:0' shape=(2, 3) dtype=float32_ref>)

In [242]:
a.ml

<tf.Variable 'mlpqq/Variable_11:0' shape=(4, 3) dtype=float32_ref>

In [243]:

a.layers[1]()

inner 1 dense_1qq/  dense_1qq
dense_1qq


False

In [244]:
a.name

'mlpqq'

In [245]:
tf.global_variables()

[<tf.Variable 'V1/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'V2/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_1qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_1qq/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_2qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_2qq/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_3qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_3qq/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/Variable:0' shape=(4, 3) dtype=float32_ref>,
 <tf.Variable 'dense_3qq/Variable:0' shape=(4, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/w_1:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/b_1:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_1qq/w_1:0' shape=(2, 3) dtype

In [246]:
a()

inner 2 mlpqq_-_1/  mlpqq_-_1
mlpqq_-_1


1

In [247]:
a.get_updates(set([e]))

get_updates {<__main__.Dense object at 0x0000018EE82CFEB8>}
======= mlpqq
ret: [] key updates
attr __call__
attr __class__
attr __delattr__
attr __dict__
attr __dir__
attr __doc__
attr __eq__
attr __format__
attr __ge__
attr __getattribute__
attr __gt__
attr __hash__
attr __init__
attr __init_subclass__
attr __le__
attr __lt__
attr __module__
attr __ne__
attr __new__
attr __reduce__
attr __reduce_ex__
attr __repr__
attr __setattr__
attr __sizeof__
attr __str__
attr __subclasshook__
attr __weakref__
attr _get_recursive_list
attr _ns_call_count
attr get_updates
attr layers
======= dense_0qq
ret: [] key updates
attr __call__
attr __class__
attr __delattr__
attr __dict__
attr __dir__
attr __doc__
attr __eq__
attr __format__
attr __ge__
attr __getattribute__
attr __gt__
attr __hash__
attr __init__
attr __init_subclass__
attr __le__
attr __lt__
attr __module__
attr __ne__
attr __new__
attr __reduce__
attr __reduce_ex__
attr __repr__
attr __setattr__
attr __sizeof__
attr __str__
attr __subcla

[]

In [171]:
a.updates

[]

In [199]:
class De(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super(De, self).__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    @tf.Module.with_name_scope
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        self.t = tf.Variable(tf.zeros([6]), name='t',trainable=False)
        self.tt = tf.Variable(tf.zeros([6]), name='tt')
        return tf.nn.relu(y)

In [200]:
d = De(3,2,'dddde')

In [201]:
d(tf.ones([1,3
          ]))

<tf.Tensor 'dddde_11/Relu:0' shape=(1, 2) dtype=float32>

In [202]:
d

In [203]:
d.name

'dddde'

In [204]:
d.variables

(<tf.Variable 'b_11:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'dddde_11/t:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'dddde_11/tt:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'w_11:0' shape=(3, 2) dtype=float32_ref>)

In [205]:
d(tf.ones([2,3]))

<tf.Tensor 'dddde_11/Relu_1:0' shape=(2, 2) dtype=float32>

In [206]:
d.variables

(<tf.Variable 'b_11:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'dddde_11/t_1:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'dddde_11/tt_1:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'w_11:0' shape=(3, 2) dtype=float32_ref>)

In [208]:
d.trainable_variables

(<tf.Variable 'b_11:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'dddde_11/tt_1:0' shape=(6,) dtype=float32_ref>,
 <tf.Variable 'w_11:0' shape=(3, 2) dtype=float32_ref>)